# Text-mining-applied-project
### Busqueda y Recuperacion de Informacion 
Se busca construir un sistema de busqueda y recuperacion de informacion que sea capáz de identificar informacion relavante para el usuario en grandes volumnes de información (Documentos Cientificos). Este proceso se lleva a cabo mediante la construcción de un Bag of Words, metodo utilizado para representar la información de los documentos en palabras.

In [1]:
import glob # libreria para extraer la ruta de los archivos
import re # libreria para manejos de expresiones regulares
import collections #### para poder contar los hash
import pandas as pd # libreria para manejo de bases datos
import numpy as np # libreria para manejo de vectores y arreglos
import json #libreria para leer los metadatos guardados como un json
import operator #Libreria para organizar de mayor a menor
from nltk.corpus import stopwords, wordnet # importa las stop words y las palabras del ingles
from nltk.stem.porter import PorterStemmer # metodo para stemming
from nltk.stem.lancaster import LancasterStemmer # metodo para stemming
from nltk.stem import WordNetLemmatizer # metodo para lematizar

### Creación del Bag of Words
Para la creacion del Bag of Words se construye una lista de todos los documentos cientificos almacenados en formato txt, luego se extrae el texto de cada uno de los documentos y se realiza una limpieza para cada una de las palabras en los diferentes documentos. Entre los procesos de limpieza estan:
* Tokenización: Es el proceso de separa un texto en cadenas separadas por espacios o signos de puntuacion.
* Eliminar Stopwords: Eliminar palabras como artículos, conjunciones y preposiciones.
* Stemming: Eliminar los sufijos de la palabra usando PorterStemmer dela libreria NLTK.
* Lemmatization: Proceso de convertir la palabra en la raiz
* Otros: Limpieza de palabras de 1 caracter y palabras pertenecientes al ingles

La estructura de datos que almancena el Bag of Words es ```[:documento][:palabra][:propiedades] ```

In [2]:
#Read Files
files_txt = glob.glob("/opt/datasets/mcda-pi1-20191/papers-txt/*.txt")

# instanciar la clase para lematizar
wordnet_lemmatizer = WordNetLemmatizer()
# llamamos al diccionario de stop words en ingles
sw = stopwords.words("english")

#leer Meta Datos
meta_data = open("xml_parser/metadata_dict.txt","r",encoding='utf-8').read()
meta_data = json.loads(meta_data)

# Recorre los archivos para generar el bag of words
bag_of_words = {} # creacion de la estructura de datos (diccionario) para almacenar el bag of words
for file in files_txt:
    #Leer Informacion
    input_file = open(file,"r",encoding='utf-8')
    texto = input_file.read()
    meta_data_info = meta_data[file.split("/")[5].replace(".txt","")]
    
    #limpieza de palabras
    texto = re.sub('[^A-Za-z0-9]+',' ',texto) # solo permanecen los elementos alfanumericos
    tokens = texto.split() # Realiza la tokenizacion
    
    meta_data_info = re.sub('[^A-Za-z0-9]+',' ',meta_data_info) # solo permanecen los elementos alfanumericos
    tokens_metada = meta_data_info.split() # Realiza la tokenización
    
    stemmer = PorterStemmer() # instancia una forma de stemming
    # aplica lematizacion, stemming, elimina de stop words y aplica reglas lógicas para reducir la cantidad de tokens
    tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and (w not in sw) and wordnet.synsets(w) and w.isalpha() ] # Longitud mayor a 1
    tokens_metada = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and (w not in sw)  ]
    
    # realiza el conteo de la frecuencia de cada palabra para cada documento
    counter = collections.Counter(tokens + tokens_metada)
    # para un diccionario almacenamos el key que es el nombre del documento y el value que es un diccionario con las key como palabras y los value como la frecuencia de la palabra
    bag_of_words[file] = dict(counter)


### Modelos
Dentro de los métodos para recuperar información de los textos, existen varios tipos de modelos. Por el momento, los modelos que están implementados para la recuperación de información son:
1.	Term Frequency (TF): Es la técnica más simple para reconocer la relevancia de un término dentro de un texto. Básicamente realiza el conteo de la palabra en el texto y mientras más grande sea este número más relevante es
2.	Relative Term Frequency (RTF): Está técnica vuelve relativo al número de palabras total el conteo anterior, representando entonces cuánto porcentaje del texto está explicado por esa palabra
3.	T-Term Frequency (T-TF): Para eliminar riesgos de modelo cuando se realiza el conteo lineal de la frecuencia de los términos, se propone trabajar con una transformación del conteo: T-TF = 1 + log(x) de esta manera no se benefician aquellas palabras que aparecen muchas veces en un documento, pues no necesariamente son más relevantes que las demás
4.	Inverse Document Frequency (IDF): Este modelo está basado en el principio de que mientras menor sea la frecuencia de la palabra en el documento, más relevante y más información puede tener IDF = log(# total de documentos/# de documentos donde está la palabra)
5.	TF-IDF: Al tener los modelos ya cuantificados, tanto el TF (recomendable trabajar con la transformación) y el IDF, la multiplicación de ambos entrega información valiosa de cara a la similaridad de la búsqueda o Query con el documento.


In [3]:
#Contar numero de documentos donde se encuentra la palabra
helper = {}
for doc, words in bag_of_words.items():
    for word in words.keys():
        if word in helper:
            helper[word] += 1
        else:
            helper[word] = 1

#Para las palabras de cada documento asignar medidas
for documento, words in bag_of_words.items():
    num_words = sum(bag_of_words[documento].values())
    for key, value in words.items():
        dic = {}
        # Asignar medidas
        dic['freq'] = value # TF
        dic['freqR'] = value / num_words # RTF
        dic['tf'] = 1 + np.log(value) # T-TF
        dic['idf'] = np.log(len(bag_of_words) / helper[key]) # IDF
        dic['tf-idf'] = dic['tf'] * dic['idf'] # TF-IDF
        bag_of_words[documento][key] = dic


In [4]:
bag_of_words['/opt/datasets/mcda-pi1-20191/papers-txt/1508.04417.txt']['user']

{'freq': 170,
 'freqR': 0.022966765738989463,
 'idf': 1.0704414117014134,
 'tf': 6.135798437050262,
 'tf-idf': 6.568012740871408}

## Primer acercamiento a Busquedas 
Se debe realizar el mismo proceso de limpieza al query ya que con este se van a buscar las palabras en el Bag of Words para buscar y identificar los textos mas relevantes 

In [5]:
#query
def transformar_informacion (texto):
    texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
    texto = texto.replace('í','i')
    tokens = texto.split()
    stemmer = PorterStemmer()
    stemmer2 = LancasterStemmer()
    tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw and wordnet.synsets(w)] # Longitud mayor a 1
    counter=collections.Counter(tokens)
    return dict(counter)

texto = "Moving average are not good for trading"
query = transformar_informacion(texto)
query

{'averag': 1, 'good': 1, 'move': 1, 'trade': 1}

### Realizar Busequeda
Despues de realizar la transformación del query utilizando los mismos proceso de limpieza que el Bag of Words, se busca cada palabra en el query para cada documento y se calcula ```[:palabra_query][:freq] * [:documento][:palabra][:tf]```

In [6]:
result_query = {}
for documento, words in bag_of_words.items():
    for key, value in words.items():
        if key in query:
            if documento in result_query:
                result_query[documento] += query[key] * value['tf']
            else:
                result_query[documento] = query[key] * value['tf']

                
resultado = sorted(result_query.items(), key=operator.itemgetter(1), reverse=True)
print("Son {} resultados.... {}".format(len(resultado), resultado[:5]))

Son 813 resultados.... [('/opt/datasets/mcda-pi1-20191/papers-txt/1106.1445.txt', 21.92645261653361), ('/opt/datasets/mcda-pi1-20191/papers-txt/1110.2053.txt', 17.136029568168144), ('/opt/datasets/mcda-pi1-20191/papers-txt/1411.4097.txt', 15.464071298752708), ('/opt/datasets/mcda-pi1-20191/papers-txt/1508.03891.txt', 15.276088586524017), ('/opt/datasets/mcda-pi1-20191/papers-txt/1511.02476.txt', 15.252080046472008)]


## Explorando Bag of Words

In [7]:
#Contar documentos
len(bag_of_words)

980

In [8]:
#Contar numero de palabras por documento despues de limpieza
x_doc = [] # documentos
y_doc = [] # num palabras unicas por doc
total_palabras_doc = [] # total palabras
for documento, words in bag_of_words.items():
    x_doc.append(documento)
    y_doc.append(len(words))
    numero_palabras = 0
    for word, values in words.items():
        numero_palabras += values['freq']
    total_palabras_doc.append(numero_palabras)

#Imprimir Solo un documento
print("El documento '{}' tiene {} palabras unicas despues del proceso de limpieza".format(x_doc[0][-15:], y_doc[0]))
print("El total de palabras contenido en el texto es de {}".format(total_palabras_doc[0]))

El documento '/1511.06030.txt' tiene 748 palabras unicas despues del proceso de limpieza
El total de palabras contenido en el texto es de 6331


In [9]:
palabras_raras = collections.OrderedDict()
palabras_tres_caracteres = collections.OrderedDict()
i = 0
for documento, words in bag_of_words.items():
    for word, values in words.items():
        if len(word) > 10 and values['freq'] < 2:
            palabras_raras[i] = word
            i+=1
        if bool(re.search(r'((\w)\2{2,})', word)):
            palabras_tres_caracteres[word] = values['idf']
palabras_raras

OrderedDict([(0, 'characteris'),
             (1, 'multidimension'),
             (2, 'multidimens'),
             (3, 'characteris'),
             (4, 'anticlockwis'),
             (5, 'anticlockwi'),
             (6, 'metamorphos'),
             (7, 'environment'),
             (8, 'unidimension'),
             (9, 'multidimension'),
             (10, 'multidimens'),
             (11, 'environment'),
             (12, 'multidimension'),
             (13, 'multidimens'),
             (14, 'anticlockwis'),
             (15, 'anticlockwi'),
             (16, 'environment'),
             (17, 'unidimension'),
             (18, 'environment'),
             (19, 'quadrilater'),
             (20, 'counterclockwis'),
             (21, 'counterclockwi'),
             (22, 'transcendent'),
             (23, 'multidimension'),
             (24, 'environment'),
             (25, 'multidimens'),
             (26, 'characteris'),
             (27, 'development'),
             (28, 'multidimension'

In [10]:
palabras_tres_caracteres

OrderedDict([('www', 0.9986746133317366),
             ('iii', 0.7263452499695411),
             ('viii', 2.793208009442517),
             ('sss', 4.489657298866247),
             ('lsss', 6.887552571664617),
             ('xviii', 5.095793102436563),
             ('aaa', 4.402645921876617),
             ('xxx', 4.248495242049359),
             ('ccc', 4.402645921876617),
             ('ppp', 5.095793102436563),
             ('ssss', 6.194405391104672),
             ('xiii', 4.584967478670572),
             ('xxviii', 6.887552571664617),
             ('kkk', 6.194405391104672),
             ('xxiii', 6.887552571664617),
             ('ysss', 6.887552571664617),
             ('yssss', 6.887552571664617),
             ('xxxi', 6.194405391104672),
             ('xxxiii', 6.887552571664617),
             ('tsss', 6.887552571664617),
             ('aisss', 6.887552571664617),
             ('xxxiv', 6.887552571664617)])

#### Explorando Bag of Words

In [18]:
#Buscar documentos con X palabra
#palabras
documentos_raros = []
for documento, words in bag_of_words.items():
    if 'bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb' in words:
        documentos_raros.append(documento)
documentos_raros

[]

In [20]:
#/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt
bag_of_words['/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt']['user']

input_file = open('/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt',"r",encoding='utf-8')
texto = input_file.read()
#texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
tokens = texto.split()
stemmer = PorterStemmer()
stemmer2 = LancasterStemmer()
tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw] # Longitud mayor a 1
counter=collections.Counter(tokens)
dict(counter)['user']

1

In [ ]:
#Traductor
def detect_language(text):

    languages_ratios = {}
    tokens = tokens = texto.split()
    tokens = [w.lower() for w in tokens]
    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(tokens)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios[language] = len(common_elements) # language "score"
    return max(languages_ratios, key=languages_ratios.get)


#----------------------------------------------------------------------
input_file = open(files_txt[0],"r",encoding='utf-8')
texto = input_file.read()
language = detect_language(texto)
language

In [11]:
bag_of_words['/opt/datasets/mcda-pi1-20191/papers-txt/1508.04417.txt']

{'social': {'freq': 186,
  'freqR': 0.025128343690894353,
  'idf': 1.5743465926228302,
  'tf': 6.225746673713202,
  'tf-idf': 9.801483062293299},
 'influenc': {'freq': 134,
  'freqR': 0.01810321534720346,
  'idf': 1.4759065198095778,
  'tf': 5.897839799950911,
  'tf-idf': 8.704660213539965},
 'concurr': {'freq': 18,
  'freqR': 0.002431775195893002,
  'idf': 2.43320527541111,
  'tf': 3.8903717578961645,
  'tf-idf': 9.466073084623341},
 'diffus': {'freq': 72,
  'freqR': 0.009727100783572008,
  'idf': 2.7131653017689805,
  'tf': 5.276666119016055,
  'tf-idf': 14.316467423134352},
 'inform': {'freq': 232,
  'freqR': 0.03134288030262091,
  'idf': 0.16492277680916945,
  'tf': 6.44673737166631,
  'tf-idf': 1.0632138286946544},
 'behavior': {'freq': 258,
  'freqR': 0.03485544447446636,
  'idf': 0.871395411966264,
  'tf': 6.552959584921617,
  'tf-idf': 5.710218917101051},
 'onlin': {'freq': 34,
  'freqR': 0.0045933531477978925,
  'idf': 1.167240795057206,
  'tf': 4.526360524616162,
  'tf-idf': 